In [63]:
import sys 
import sqlite3
import argparse
import os 
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

True

In [64]:
# We need to create a connection to the database file, named vocab.db
# We have retrieved this file from our kindle, and it contains the words we have looked up
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except sqlite3.Error as e:
        print(e)
    return conn

In [65]:
# We want to joint the two tables from LOOKUPS, BOOK_INFO and WORDS
# We can do this by making use of the book_key from LOOKUPS and id from BOOK_INFO
# To join the other tables we have to make use of the word_key from LOOKUPS and id from WORDS
# The only columns we need are: word_key/id, word, stem, title, and usage
def join_tables(conn):
    cur = conn.cursor()
    cur.execute("SELECT word_key, word, title, usage FROM LOOKUPS JOIN WORDS ON LOOKUPS.word_key = WORDS.id JOIN BOOK_INFO ON LOOKUPS.book_key = BOOK_INFO.id")
    rows = cur.fetchall()
    return rows

In [66]:
# Let's try our code
def retrieve_rows(file):
    conn = create_connection(file)
    with conn:
        rows = join_tables(conn)
        return rows

In [ ]:
main()

In [68]:
# Let's create an OpenAI client to use the API
client = OpenAI()

In [81]:
# For each word in our joined tables, we want to determine the meaning of the word in the context of the usage. 
# We will use the OpenAI client to generate a response for each word
def generate_definition(word, usage):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            #{"role": "system", "content": "You are a language bot who ONLY returns the English definition of the word given the context. The translation must at most be several words."},
            {"role": "system", "content": "You are a language bot who EXCLUSIVELY returns the English definition of the word given the context, without explanation. The translation must at most be several words."},
            {
                "role": "user",
                "content": f"Define \"{word}\" in the context of \"{usage}\"."
            }
        ]
    )
    return completion.choices[0].message


In [79]:
# # Take first 10 rows for testing
# rows = main()
# rows = rows[:50]
# responses = []
# for row in rows:
#     word = row[1]
#     usage = row[3]
#     response = generate_definition(word, usage)
#     responses.append(response)

# Take first 10 rows for testing
rows = main()
rows = rows[:50]
responses = []
results = []
for row in rows:
    word = row[1]
    usage = row[3]
    response = generate_definition(word, usage)
    responses.append(response)


In [80]:
responses

[ChatCompletionMessage(content='"I"', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='to deceive or fool', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='time', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='Morning light', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='welcome', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='remaining', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='self-criticism', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='"Stop it."', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='Bustling or flourishing world.', refusal=None, rol